## Logistic Regression on TF-IDF
### Training Pipeline

In [43]:
# !pip install dagshub

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import *

import dagshub
import pickle

In [59]:
def load_corpus(DATASET_PATH, CODE_COLUMN):
    df = pd.read_csv(DATASET_PATH, encoding='utf-8', comment='#', sep=',')#, quoting=csv.QUOTE_NONE, error_bad_lines=False)#, sep=','
    print(df.head())
    corpus = df[CODE_COLUMN]
    test_size = 0.1
    test_rows = round(df.shape[0]*test_size)
    train_rows = df.shape[0] - test_rows
    train_corpus = df[CODE_COLUMN][0:test_rows]
    test_corpus = df[CODE_COLUMN][train_rows:]
    return df, corpus

In [60]:
def tfidf_transform(corpus, tfidf_params, TFIDF_DIR):
#     tfidf = TfidfVectorizer(min_df=5
#                             , max_df = 0.3
#                             , ngram_range = (1,2)
#                             , smooth_idf = True
#                            )
    # tfidf = TfidfVectorizer(tfidf_params)
    # features = tfidf.fit_transform(corpus)
    tfidf = pickle.load(open(TFIDF_DIR, 'rb'))
    features = tfidf.transform(corpus)
    return features

In [61]:
def tfidf_fit_transform(code_blocks, tfidf_params, TFIDF_DIR):
    tfidf = TfidfVectorizer(tfidf_params)
    tfidf = tfidf.fit(code_blocks)
    pickle.dump(tfidf, open(TFIDF_DIR, "wb"))
    code_blocks_tfidf = tfidf.transform(code_blocks)
    return code_blocks_tfidf

In [62]:
def logreg_evaluate(df, code_blocks, TAG_TO_PREDICT):
    code_blocks_tfidf = tfidf_fit_transform(code_blocks, tfidf_params, TFIDF_DIR)
    X_train, X_test, y_train, y_test = train_test_split(code_blocks_tfidf, df[TAG_TO_PREDICT], test_size=0.25)
    clf = LogisticRegression(random_state=421).fit(X_train, y_train)
    # clf.fit(X_train, y_train)
    print("saving the model")
    pickle.dump(clf, open(MODEL_DIR, 'wb'))
    y_pred = clf.predict(X_test)
    accuracy = clf.score(X_test, y_test)
    f1 = f1_score(y_pred, y_test)
    print(f'Mean Accuracy {round(accuracy*100, 2)}%')
    print(f'F1-score {round(f1*100, 2)}%')
    errors = y_test - y_pred
    plt.hist(errors)
    plot_precision_recall_curve(clf, X_test, y_test)
    plot_confusion_matrix(clf, X_test, y_test, values_format='d')
    def mean_confidence_interval(data, confidence=0.95):
        a = 1.0 * np.array(data)
        n = len(a)
        m, se = np.mean(a), scipy.stats.sem(a)
        h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
        return m, m-h, m+h
    conf_interval = mean_confidence_interval(errors, 0.95)
    print(conf_interval)
    metrics = {'test_accuracy': accuracy
               , 'test_f1_score': f1}
    return metrics

In [63]:
def get_predictions(X, y, TAG_TO_PREDICT, MODEL_DIR):
    clf = pickle.load(open(MODEL_DIR, 'rb'))
    # result = loaded_model.score(X, y)
    y_pred = clf.predict(X)
    accuracy = accuracy_score(y_pred, y)
    f1 = f1_score(y_pred, y, average='weighted')
    print(f'Mean Accuracy {round(accuracy*100, 2)}%')
    print(f'F1-score {round(f1*100, 2)}%')
    errors = y - y_pred
    plt.hist(errors)
    plot_precision_recall_curve(clf, X, y)
    plot_confusion_matrix(clf, X, y, values_format='d')
    def mean_confidence_interval(data, confidence=0.95):
        a = 1.0 * np.array(data)
        n = len(a)
        m, se = np.mean(a), scipy.stats.sem(a)
        h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
        return m, m-h, m+h
    conf_interval = mean_confidence_interval(errors, 0.95)
    print(conf_interval)
    metrics = {'test_accuracy': accuracy
               , 'test_f1_score': f1}
    return metrics

### Constants

In [64]:
GRAPH_VER = 3
DATASET_PATH = './data/code_blocks_regex_graph_v{}.csv'.format(GRAPH_VER)
MODEL_DIR = './models/logreg_regex_graph_v{}.sav'.format(GRAPH_VER)
TFIDF_DIR = './models/tfidf_logreg_graph_v{}.pickle'.format(GRAPH_VER)
CODE_COLUMN = 'code_block'
TAG_TO_PREDICT = 'preprocessing'
PREDICT_COL = 'pred_{}'.format(TAG_TO_PREDICT)
SCRIPT_DIR = 'logreg_classifier.ipynb'

VAL_CHUNK_SIZE = 10
VAL_CODE_COLUMN = 'code'
VAL_TAG_TO_PREDICT = 'tag'
VAL_DATASET_PATH = './data/chunks_{}_validate.csv'.format(VAL_CHUNK_SIZE)

### Evaluation

In [58]:
if __name__ == '__main__':
    df, code_blocks = load_corpus(DATASET_PATH, CODE_COLUMN)
    nrows = df.shape[0]
    print("loaded")
    tfidf_params = {'min_df': 5
                    , 'max_df': 0.3
                    , 'smooth_idf': True}
    data_meta = {'DATASET_PATH': DATASET_PATH
                ,'nrows': nrows
                ,'label': TAG_TO_PREDICT
                ,'model': MODEL_DIR
                ,'script_dir': SCRIPT_DIR}
    print("tfidf-ed")
    with dagshub.dagshub_logger() as logger:
        metrics = logreg_evaluate(df, code_blocks, TAG_TO_PREDICT)
        # metrics = get_predictions(features, df[TAG_TO_PREDICT], TAG_TO_PREDICT, MODEL_DIR)
        logger.log_hyperparams(data_meta)
        logger.log_hyperparams(tfidf_params)
        logger.log_metrics(metrics)
    print("finished")

index,code_block,tag,imports,data_load,preprocessing,visualization,model,train,predict
0                                                0,"                                    
1                                import pandas as pd                                    
2                    import matplotlib.pyplot as plt                                    
3                              import seaborn as sns                                    
4                                 import numpy as np                                    


KeyError: 'code_block'

## Errors Analysis

In [52]:
def analyze_predictions(X, y, TAG_TO_PREDICT, MODEL_DIR):
    clf = pickle.load(open(MODEL_DIR, 'rb'))
    # result = loaded_model.score(X, y)
    y_pred = clf.predict(X)
    accuracy = accuracy_score(y_pred, y)
    f1 = f1_score(y_pred, y, average='weighted')
    print(f'Mean Accuracy {round(accuracy*100, 2)}%')
    print(f'F1-score {round(f1*100, 2)}%')
    errors = y - y_pred
    plt.hist(errors)
    plot_precision_recall_curve(clf, X, y)
    plot_confusion_matrix(clf, X, y, values_format='d')
    def mean_confidence_interval(data, confidence=0.95):
        a = 1.0 * np.array(data)
        n = len(a)
        m, se = np.mean(a), scipy.stats.sem(a)
        h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
        return m, m-h, m+h
    conf_interval = mean_confidence_interval(errors, 0.95)
    print(conf_interval)
    metrics = {'test_accuracy': accuracy
               , 'test_f1_score': f1}
    return X, y, y_pred
if __name__ == '__main__':
    df, corpus = load_corpus(VAL_DATASET_PATH, VAL_CODE_COLUMN)
    nrows = df.shape[0]
    print("loaded")
    params = {'min_df': 5
             , 'max_df': 0.3
             , 'smooth_idf': True}
    features = tfidf_transform(corpus, params, TFIDF_DIR)
    print("tfidf-ed")
    _, _, y_pred = analyze_predictions(features, df[VAL_TAG_TO_PREDICT], VAL_TAG_TO_PREDICT, MODEL_DIR)
    print("finished")
    df[PREDICT_COL] = y_pred

code  tag
0      dataset['Embarked'] = dataset['Embarked']....    1
1  for dataset in combine:\n    dataset['Title'] ...    1
2      dataset['Title'] = dataset['Title'].fillna...    1
3          dataset[entry] = np.log1p(dataset[entr...    1
4  sc_X = StandardScaler()\nX_train = sc_X.fit_tr...    1
loaded


FileNotFoundError: [Errno 2] No such file or directory: './models/tfidf_logreg_graph_v3.pickle'

In [53]:
# pd.set_option('max_colwidth', 500)
# pd.set_option('max_rows', 500)

In [54]:
## True Negatives
df[(df[VAL_TAG_TO_PREDICT] == 0)&(df[PREDICT_COL] == 0)][VAL_CODE_COLUMN].to_csv('./model_interpretation/errors_analysis/logreg_validation_{}_chunksize_{}_conf_matrix_TN.txt'.format(GRAPH_VER, VAL_CHUNK_SIZE), index=False)

KeyError: 'pred_preprocessing'

In [55]:
# ## False Positives (preprocessing y/n)
# corpus[(y_pred-y)==-1].reset_index(drop=True)[:]

## Weights Analysis

In [56]:
# from operator import itemgetter
# def show_most_informative_features(model, vectorizer=None, text=None, n=20):
#     """
#     Extract the vectorizer and the classifier from the pipeline
#     """
#     if vectorizer is None:
#         vectorizer = model.named_steps['vectorizer']
#     else:
#         text = vectorizer.transform([text])

#     classifier = model#.named_steps['classifier']
#     feat_names = vectorizer.get_feature_names()

#     # Check to make sure that we can perform this computation
#     if not hasattr(classifier, 'coef_'):
#         raise TypeError(
#             "Cannot compute most informative features on {}.".format(
#                 classifier.__class__.__name__
#             )
#         )    

#     # Otherwise simply use the coefficients
#     tvec = classifier.coef_

#     # Zip the feature names with the coefs and sort   
#     coefs = sorted(
#         zip(tvec[0], feat_names),
#         key=itemgetter(0), reverse=True
#     )

#     # Get the top n and bottom n coef, name pairs
#     topn  = zip(coefs[:n], coefs[:-(n+1):-1])

#     # Create the output string to return
#     output = []
#     # output = pd.DataFrame()
#     # If text, add the predicted value to the output.
#     if text is not None:
#         # output.append("\"{}\"".format(text))
#         # output.append(
#         #     "Classified as: {}".format(model.predict(text))
#         # )
#         # output.append("")
#         print("Classified as: {}".format(model.predict(text)))
#     # Create two columns with most negative and most positive features.

#     for (cp, fnp), (cn, fnn) in topn:
#         print(cp, fnp, cn, fnn)
#         output.append(
#             "{:0.2f}{: >15}    {:0.2f}{: >15}".format(
#                 cp, fnp, cn, fnn
#             )
#         )

#     return "\n".join(output)

In [57]:
# MODEL_DIR = './models/logreg_regex_graph_v2.2.sav'
# TFIDF_DIR = './models/tfidf_logreg_graph_v2.2.pickle'
# DATASET_PATH = './data/chunks_5_validate.csv'
# CODE_COLUMN = 'code'
# TAG_TO_PREDICT = 'tag'
# df, corpus = load_corpus(DATASET_PATH, CODE_COLUMN)
# interpret = show_most_informative_features(model=pickle.load(open(MODEL_DIR, 'rb')),
#                                 vectorizer=pickle.load(open(TFIDF_DIR, 'rb')),
#                                 text=corpus[5],
#                                 n = 500)
# # with open('logreg_interpret.txt', mode='w') as f:
# #     f.write(interpret)